Program that will prepare fits images prepared with my calibration scripts for Paul's geo cal code. Changes header keywords, calculates and attaches the North Pole position angle given a tcc log file from the telescope. Also makes a textfile with the names of images and their location to be used as input for disk_corr.pro.

In [ ]:
import numpy as np
from astropy.io import fits
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [ ]:
#program that finds north polar position angle
def rot_angle(directory,images,apolog):
    
    #directory: directory where data is
    #images: name of image list
    #apolog: location and name of date_tel.txt files
    
    image_list = np.loadtxt(images,dtype=str)
    apo_log = np.loadtxt(apolog,dtype=str)
    
    #Make arrays of the log's alt, az, and JD:
    tcc_rot_ang = []
    jd = []
    
    JD_per_s = 1./86400
    JD_per_min = 1./1440
    JD_per_hr = 1./24
    
    #Find the jd of the night; pull it from an image in the list
    juliandate = float(int(fits.open(directory+image_list[1])[0].header['julian']))
    
    #make lists of rotation angles from apo log to make into functions:
    for i in range(0,len(apo_log)):
        #THIS ASSUMES THAT THE TELESCOPE IS RECORDING +CW until 180, then negative from ccw
        #append the value of ObjInsAng if it's positive
        if float(apo_log[i][12][11:-1]) >= 0:
            tcc_rot_ang.append(float(apo_log[i][12][11:-1]))
        #if it's negative:
        if float(apo_log[i][12][11:-1]) < 0:
            tcc_rot_ang.append(360.0+float(apo_log[i][12][11:-1]))
        
        #Convert hours:minutes:seconds into JD, append it
        jd.append(juliandate+0.5+(float(apo_log[i][1][0:2])*JD_per_hr)+(float(apo_log[i][1][3:5])*JD_per_min)+(float(apo_log[i][1][6:-1])*JD_per_s))
    
    #Make functions of the rot angle as a function of JD via interp1d
    rotang_func = interp1d(jd,tcc_rot_ang)
    
    #return jd,tcc_rot_ang

    #Open the image and find the JD
    for i in range(0,len(image_list)):
        
        im = fits.open(directory+image_list[i])
        image = im[0].data
        header = im[0].header
        if header['rfon'] == 1:
            im_jd = header['julian']
            #print 'Appending quantities for image '+image_list[i]+' taken at '+str(im_jd)+'...'

            im_rotang = round(rotang_func(im_jd),5)

            #Append the alt and az of the instrument at the image's JD
            header['ROT_ANG'] = (im_rotang,'(Degrees) angle cw from pos vertical to NCP')

            #Find np_ang
            npa = round(im_rotang + header['NPPANGLE'],5)
            header['PNORTH'] = (npa,'Angle of the planets NP cw from CN (Degrees)')

            #Save image:
            im.writeto(directory+image_list[i],overwrite=True)
            im.close()

In [ ]:
def geocal_preparer(directory,imagelist,tccfile,directory2,pixmask,directory_pixmask):
    '''
    directory - location of images to edit
    imagelist - name and path of list of science images
    tccfile - name of telescope file
    directory2 - name of directory (probably on desktop) that will be used in the generated list for input to disk_corr.pro
    pixmask - the name of the ASCII pixel mask
    directory_pixmask - location of pixel mask
    '''
    
    camera_query = raw_input('Was this image cube taken before Jan. 2018 (before the camera replacement)? If y, keywords APO will be changed to APO1. If n, keyword APO will be changed to APO2. Pauls program ORTHO_MAP.pro looks for these keywords to designate plate scale and pixel size, which are different for each camera.')
    
    #load image list
    imlist = np.loadtxt(imagelist,dtype=str)
    rfon_list = []
    
    #list of keywords to change:
    old_keyword_list = ['OBS_LAT','OBS_LONG','SOL_LAT','SOL_LONG','OBS_R']
    new_keyword_list = ['SE_LAT','SE_LON','SS_LAT','SS_LON','PE_RNG']
    
    #find and attach the np position angle
    print 'Finding NP position angle...'
    rot_angle(directory,imagelist,tccfile)
    
    
    # Making pixel mask into a fits file
    
    ar = np.loadtxt(directory_pixmask+pixmask,dtype=int)
    hdu = fits.PrimaryHDU(ar)
    hdul = fits.HDUList([hdu])
    hdul[0].header['History'] = 'Pixel mask made from '+pixmask
    hdul.writeto(directory_pixmask+pixmask+'.fits',overwrite=True)
    
    
    #Rename keywords
    #print 'Renaming keywords...'
    print 'Making RF-on list, updating APO keyword...'
    for i in range(0,len(imlist)):
        im = fits.open(directory+imlist[i])
        if im[0].header['rfon'] == 1:
            rfon_list.append(imlist[i])
            #Indicating which camera was used
            if camera_query == 'y':
                im[0].header['OBSERVAT'] = ('APO1','APO1-taken w/ old cam')
            if camera_query == 'n':
                im[0].header['OBSERVAT'] = ('APO2','APO2-taken w/ new cam')
            #Saving the name of the pixel mask
            im[0].header['PIX_MASK'] = (pixmask+'.fits','Name of pixel mask file')
        im.writeto(directory+imlist[i],overwrite=True)
        im.close()           
    #        for j in range(0,len(old_keyword_list)):
    #            im[0].header.rename_keyword(old_keyword_list[j],new_keyword_list[j])
    #    #save the image and close it
    #    im.writeto(directory+imlist[i],overwrite=True)
    #    im.close()

        
    #make a list with images' paths to be used as input
    print 'Making input file for corr_disk.pro...'
    x = []
    for k in range(0,len(rfon_list)):
        x.append(str(directory2+rfon_list[k]))
    np.savetxt(directory+'input_list',x,fmt='%s')

In [ ]:
geocal_preparer('/Volumes/external_hd/march_2017_data/jupiter_2/',\
                '/Users/dahlek/Desktop/march2017/march_2017_sciimages',\
                '/Users/dahlek/Desktop/march2017/170327_tel.txt',\
                '/home/users/dahlek/march2017_geocal/',\
                'pix_mask_march2017',\
                '/Users/dahlek/Desktop/march2017/')